In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import openai
from IPython.display import Markdown, display, HTML

os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from src.doc_store.utils import download_mp3_tempfile

In [4]:
import re
from typing import List, Optional, Union
from pydantic import BaseModel, Field, HttpUrl, conint, field_validator, model_validator
from datetime import datetime

from src.doc_store.courtlistener import COURTLISTENER_WEB_URL
from src.schema.audio import Source, SOURCES

class AudioResource(BaseModel):
    resource_uri: HttpUrl
    id: int
    frontend_url: HttpUrl
    absolute_url: str
    panel: List[str] = []
    docket: HttpUrl
    date_created: datetime
    date_modified: datetime
    source: Source
    case_name_short: str
    case_name: str
    case_name_full: Optional[str] = ""
    judges: Optional[str] = ""
    sha1: str
    download_url: HttpUrl
    local_path_mp3: HttpUrl
    local_path_original_file: str
    filepath_ia: HttpUrl
    duration: int
    processing_complete: bool
    date_blocked: Optional[datetime] = None
    blocked: bool
    stt_status: conint(ge=0)
    stt_google_response: Optional[str] = ""
    
    @model_validator(mode="before")
    def generate_frontend_url(cls, values):
        absolute_url = values.get('absolute_url')
        if absolute_url:
            base_url = COURTLISTENER_WEB_URL
            values['frontend_url'] = f"{base_url}{absolute_url}"
        return values
    
    @model_validator(mode="before")
    def generate_mp3_url(cls, values):
        local_path_mp3 = values.get('local_path_mp3')
        if local_path_mp3:
            base_url = "https://www.courtlistener.com/"
            values['local_path_mp3'] = f"{base_url}{local_path_mp3}"
        return values
    
    @property
    def whisper_cost(self):
        audio_duration_in_minutes = self.duration / 60
        return round(audio_duration_in_minutes, 2) * 0.006
    
    def __str__(self) -> str:
        """
        Generates a string representation of the object, combining various attributes.

        Returns:
            str: A string representation of the object.
        """
        text_parts = []
        case_id = f"Case ID: {self.docket_num}"
        text_parts.append(case_id)
        source = self.get_source_description
        source_str = f"Source: [{source}]({self.download_url})"
        text_parts.append(source_str)
        name_short = str(self.case_name)
        text_parts.append(name_short)
        date_argued_str = self.serialize_date(self.date_argued)
        date = f"Date Argued: {date_argued_str[:10]}"
        text_parts.append(date)
        court = f"Court: {str(self.court)}"
        text_parts.append(court)
        cleaned_snippet = re.sub(r'\n+', '\n', self.snippet).strip()
        text_parts.append(cleaned_snippet)
        return "\n\n".join(text_parts)

   
class OralArgumentSearchResult(BaseModel):
    absolute_url: str
    frontend_url: HttpUrl
    case_name: str = Field(
        ...,
        alias="caseName",
    )
    court: str
    court_citation_string: str
    court_exact: str
    court_id: str
    date_argued: datetime = Field(
        ...,
        alias="dateArgued"
    )
    docket_num: str = Field(
        ...,
        alias="docketNumber"
    )
    docket_id: int
    download_url: HttpUrl
    duration: int
    file_size_mp3: int
    id: int
    judge: Optional[str] = ''
    local_path: str
    pacer_case_id: Optional[int] = None
    panel_ids: Optional[int | List[int]] = None
    sha1: Optional[str] = None
    snippet: str
    source: Source
    timestamp: datetime
    
    @property
    def get_source_description(self) -> Optional[str]:
        """
        Returns the description of the current source enum value.

        Returns:
            Optional[str]: The description of the selected source, or None if no source is selected.
        """
        if self.source is None:
            return None
        # Find the description in SOURCES.NAMES using the current source value
        for code, description in SOURCES.NAMES:
            if code == self.source.value:
                return description
        return None
    
    @model_validator(mode="before")
    def generate_frontend_url(cls, values):
        absolute_url = values.get('absolute_url')
        if absolute_url:
            base_url = COURTLISTENER_WEB_URL
            values['frontend_url'] = f"{base_url}{absolute_url}"
        return values
    
    @field_validator("date_argued", mode="before")
    def serialize_date(cls, v: Union[datetime, str]) -> str:
        """
        Serializes the date_argued field to a string format.

        Args:
            date_argued (Union[date, str]): The decision date to be serialized.

        Returns:
            str: The serialized decision date as a string.
        """
        if isinstance(v, datetime):
            return v.isoformat()
        return v
    
    @property
    def whisper_cost(self):
        audio_duration_in_minutes = self.duration / 60
        return round(audio_duration_in_minutes, 2) * 0.006
    
    def __str__(self) -> str:
        """
        Generates a string representation of the object, combining various attributes.

        Returns:
            str: A string representation of the object.
        """
        text_parts = []
        case_id = f"Case ID: {self.docket_num}"
        text_parts.append(case_id)
        source = self.get_source_description
        source_str = f"Source: [{source}]({self.download_url})"
        text_parts.append(source_str)
        name_short = str(self.case_name)
        text_parts.append(name_short)
        date_argued_str = self.serialize_date(self.date_argued)
        date = f"Date Argued: {date_argued_str[:10]}"
        text_parts.append(date)
        court = f"Court: {str(self.court)}"
        text_parts.append(court)
        # Clean snippet by replacing all sequences of multiple line breaks with a single line break
        cleaned_snippet = re.sub(r'\n+', '\n', self.snippet).strip()
        text_parts.append(cleaned_snippet)
        return "\n\n".join(text_parts)


In [5]:
import requests

from src.schema.docket import Docket

class CourtListenerAudio:
    """
    A class to interact with the CourtListener Audio API endpoint.
    """
    AUDIO_URL = 'https://www.courtlistener.com/api/rest/v3/audio/'
    SEARCH_URL = 'https://www.courtlistener.com/api/rest/v3/search/'
    
    def __init__(self):
        """
        Initializes the CourtListenerAudio with the necessary API token.

        """
        self.api_token = os.getenv("COURTLISTENER_API_KEY")
        self.headers = {'Authorization': f'Token {self.api_token}'}
        
    def get_docket(self, url: HttpUrl):
        response = requests.get(url, headers=self.headers)
        response_dict = response.json()
        return Docket(**response_dict)
    
    def get_docket_id(self, docket_id: int):
        url = f"https://www.courtlistener.com/api/rest/v3/dockets/{docket_id}/"
        response = requests.get(url, headers=self.headers)
        response_dict = response.json()
        return Docket(**response_dict)
    
    def get_audio(self, url: HttpUrl):
        response = requests.get(url, headers=self.headers)
        response_dict = response.json()
        
        return AudioResource(**response_dict)

    def query_audio(self, filters: dict = None, ordering: str = None) -> List[AudioResource]:
        """
        Queries the CourtListener Audio endpoint with optional filters and ordering.

        Args:
            filters (dict, optional): A dictionary of filters to apply to the query.
            ordering (str, optional): A string specifying the ordering of the results.

        Returns:
            AudioResource: A list of AudioResource objects.
        """
        params = filters if filters else {}
        if ordering:
            params['ordering'] = ordering

        response = requests.get(self.AUDIO_URL, headers=self.headers, params=params)
        response.raise_for_status()
        search_res = response.json()["results"]
        return [AudioResource(**s) for s in search_res]
    
    
    def search_oral_arguments(self, query: str, filters: dict = None, ordering: str = None) -> dict:
        """
        Searches for oral arguments using the CourtListener search endpoint with the type "oa".

        Args:
            query (str): The search query string.
            filters (dict, optional): Additional filters to apply to the search.
            ordering (str, optional): A string specifying the ordering of the results.

        Returns:
            dict: The JSON response from the CourtListener API.
        """
        params = {'q': query, 'type': 'oa'}
        if filters:
            params.update(filters)
        if ordering:
            params['ordering'] = ordering

        response = requests.get(self.SEARCH_URL, headers=self.headers, params=params)
        response.raise_for_status()
        search_res = response.json()["results"]
        
        return [OralArgumentSearchResult(**s) for s in search_res]


In [7]:
cl_audio = CourtListenerAudio()

# Example query: Get audio files ordered by date filed descending
# filters = {'case_name': 'Roe v. Wade'}
# ordering = '-date_created'
# results = cl_audio.query_audio(filters=filters, ordering=ordering)
# results[0].model_dump()

In [9]:
search_res = cl_audio.search_oral_arguments("Liberty Mutual", ordering = '-dateArgued')
search_res[0].model_dump()

{'absolute_url': '/audio/88046/naperville-hotel-partners-llc-v-liberty-mutual-fire-insurance-co/',
 'frontend_url': Url('https://www.courtlistener.com/audio/88046/naperville-hotel-partners-llc-v-liberty-mutual-fire-insurance-co/'),
 'case_name': 'Naperville Hotel Partners, LLC v. Liberty Mutual Fire Insurance Co.',
 'court': 'Appellate Court of Illinois',
 'court_citation_string': 'Ill. App. Ct.',
 'court_exact': 'illappct',
 'court_id': 'illappct',
 'date_argued': datetime.datetime(2023, 9, 13, 0, 0, tzinfo=TzInfo(-07:00)),
 'docket_num': '3-22-0440',
 'docket_id': 67792176,
 'download_url': Url('https://ilcourtsaudio.blob.core.windows.net/antilles-resources/resources/8df40f12-d8fd-4552-a5e2-6ff3341163b8/3-22-0440%20Naperville%20Hotel%20Partners,%20LLC%20v.%20Liberty%20Mutual%20Fire%20Insurance%20Co..mp3'),
 'duration': 1806,
 'file_size_mp3': 12631982,
 'id': 88046,
 'judge': '',
 'local_path': 'mp3/2023/09/13/naperville_hotel_partners_llc_v._liberty_mutual_fire_insurance_co._cl.mp3'

In [10]:
docket = cl_audio.get_docket_id(search_res[0].docket_id)

In [11]:
docket.model_dump()

{'resource_uri': 'https://www.courtlistener.com/api/rest/v3/dockets/67792176/',
 'id': 67792176,
 'court': 'https://www.courtlistener.com/api/rest/v3/courts/illappct/',
 'court_id': 'illappct',
 'clusters': ['https://www.courtlistener.com/api/rest/v3/clusters/9436667/'],
 'audio_files': ['https://www.courtlistener.com/api/rest/v3/audio/88046/'],
 'assigned_to': None,
 'referred_to': None,
 'absolute_url': '/docket/67792176/naperville-hotel-partners-llc-v-liberty-mutual-fire-insurance-co/',
 'date_created': datetime.datetime(2023, 9, 13, 12, 41, 46, 749798, tzinfo=TzInfo(-07:00)),
 'date_modified': datetime.datetime(2023, 11, 1, 14, 5, 57, 389531, tzinfo=TzInfo(-07:00)),
 'source': <Source.SCRAPER: 2>,
 'appeal_from_str': '',
 'assigned_to_str': '',
 'referred_to_str': '',
 'panel_str': '',
 'case_name_short': '',
 'case_name': 'Naperville Hotel Partners, LLC v. Liberty Mutual Fire Insurance Co.',
 'case_name_full': '',
 'slug': 'naperville-hotel-partners-llc-v-liberty-mutual-fire-insur

In [12]:
audio = cl_audio.get_audio(docket.audio_files[0])

In [13]:
audio.model_dump()

{'resource_uri': Url('https://www.courtlistener.com/api/rest/v3/audio/88046/'),
 'id': 88046,
 'frontend_url': Url('https://www.courtlistener.com/audio/88046/naperville-hotel-partners-llc-v-liberty-mutual-fire-insurance-co/'),
 'absolute_url': '/audio/88046/naperville-hotel-partners-llc-v-liberty-mutual-fire-insurance-co/',
 'panel': [],
 'docket': Url('https://www.courtlistener.com/api/rest/v3/dockets/67792176/'),
 'date_created': datetime.datetime(2023, 9, 13, 12, 41, 46, 751966, tzinfo=TzInfo(-07:00)),
 'date_modified': datetime.datetime(2023, 9, 14, 20, 24, 25, 810175, tzinfo=TzInfo(-07:00)),
 'source': <Source.COURT_WEBSITE: 'C'>,
 'case_name_short': '',
 'case_name': 'Naperville Hotel Partners, LLC v. Liberty Mutual Fire Insurance Co.',
 'case_name_full': '',
 'judges': '',
 'sha1': '4ca9da990e333d32086bf6e1a5de84e07cb6bad4',
 'download_url': Url('https://ilcourtsaudio.blob.core.windows.net/antilles-resources/resources/8df40f12-d8fd-4552-a5e2-6ff3341163b8/3-22-0440%20Naperville%2

In [25]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()

mp3_url = str(audio.download_url)

audio_tempfile = download_mp3_tempfile(mp3_url)

File downloaded and saved as C:\Users\pdoub\AppData\Local\Temp\tmpmn2n0ynu.mp3
File Size: 48.13 MB


In [33]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def process_and_transcribe(filepath: str, min_file_size: int = 1 * 1024 * 1024, max_file_size: int = 15 * 1024 * 1024):
    """
    Processes an MP3 file by splitting it based on silence, then dynamically combining or splitting chunks
    to ensure each is within a specified size range before transcribing. Saves the final transcription to a text file
    and returns the transcription objects.

    Args:
        filepath (str): Path to the MP3 file to be processed.
        min_file_size (int, optional): Minimum file size for each chunk in bytes. Defaults to 1MB.
        max_file_size (int, optional): Maximum file size for each chunk in bytes. Defaults to 15MB.

    Returns:
        List[dict]: A list of transcription objects returned by the transcription API.
    """
    sound = AudioSegment.from_file(filepath)
    chunks = split_on_silence(
        sound,
        min_silence_len=500,
        silence_thresh=sound.dBFS - 16,
        keep_silence=250,
    )

    transcriptions = []  # To store transcription texts
    combined_chunk = AudioSegment.empty()
    for _, chunk in enumerate(chunks):
        combined_chunk += chunk
        combined_chunk_length = len(combined_chunk)

        if combined_chunk_length >= min_file_size:
            if combined_chunk_length > max_file_size:
                # Split and transcribe large chunks
                half = combined_chunk_length // 2
                transcriptions.append(transcribe_chunk(combined_chunk[:half]))
                transcriptions.append(transcribe_chunk(combined_chunk[half:]))
                combined_chunk = AudioSegment.empty()
            else:
                # Transcribe suitable chunks
                transcriptions.append(transcribe_chunk(combined_chunk))
                combined_chunk = AudioSegment.empty()

    if len(combined_chunk) > 0:
        transcriptions.append(transcribe_chunk(combined_chunk))

    # Save transcriptions to a text file
    with open("final_transcription.txt", "w") as f:
        for transcription in transcriptions:
            f.write(transcription + "\n\n")

    return transcriptions

def transcribe_chunk(chunk: AudioSegment) -> dict:
    """
    Transcribes a given audio chunk using the OpenAI Whisper model and returns the transcription object.

    Args:
        chunk (AudioSegment): The audio chunk to be transcribed.

    Returns:
        dict: The transcription object returned by the transcription API.
    """
    temp_file = "temp_chunk.mp3"
    chunk.export(temp_file, format="mp3")

    audio_file = open(temp_file, "rb")
    # Transcribe the audio file
    transcription = client.audio.transcriptions.create(
        model="whisper-1", 
        file=audio_file,
        response_format="json",
        )
    audio_file.close()
    os.remove(temp_file)
    return transcription.text

In [34]:
scripts = process_and_transcribe(audio_tempfile)

In [35]:
len(scripts)

2

In [38]:
transcription = "\n".join(scripts)
Markdown(transcription)

322-0440, Naperville Hotel Partners, LLC, and Superhost Hospitality, Inc., Appellants versus Liberty Mutual Fire Insurance Company and Fireman's Fund Insurance Company, Appelese. Thank you. Good afternoon. Good afternoon. Good afternoon, Your Honor. Mr. Connor, are you ready to begin? Yes, I am, Your Honor. Good afternoon. Colin Connor on behalf of the Appellants. This appeal addresses three key issues in defendants' motions to dismiss. First, the internal statute of limitations does not clearly, definitively, and explicitly begin to run from the beginning of the injury rather than from the end. Second, defendants should have been stopped from asserting their internal statute of limitations defense. And third, Naperville gave adequate notice of its claim. Given the high hurdles that a motion to dismiss must overcome and Illinois law liberally construing policies in favor of the policyholder, Naperville's reasonable interpretation of the internal statute of limitations should govern. The overall considerations are the policy language and reasonableness. The insurer's policies state that a suit must be brought within two years after the date on which the physical loss or damage occurred. Defendants did not include any words like starts, incepts, or begins in their policy, however. Instead, defendants require a suit within two years after the date when any damage occurred without regard to when it began. The only decisions interpreted in this language that either party provided to the trial court find that the limitation runs from the conclusion of the loss, and both decisions involve water infiltration like Naperville has experienced. In the Panorama case, for example, the policy required the policyholder to bring suit within one year after a loss occurs. The Washington Supreme Court stated, a plain, ordinary reading of the contract suggests the policyholder must bring an action for coverage within one year subsequent to or succeeding the loss. This must be distinguished from a contract which requires a policyholder to bring a coverage action within 12 months after the inception of the loss. A second directly on point case is Straus. The policy required that suit be filed within one year after a loss occurs. The Seventh Circuit analyzed the issue the same way as Panorama, stating, after a loss occurs is fundamentally different from after the inception. Beginning the loss clearly and unmistakably means the beginning of damage, while after a loss occurs is ambiguous. Thus, the Seventh Circuit held that the Strauses could have brought their claim at any point up until after the water infiltration damage halt. As the Panorama court stated, the industry knows how to protect itself and it knows how to write exclusions and conditions. The Illinois Supreme Court has held a policyholder should not be deprived of the benefit of insurance for which he has paid, except for where the terms of the policy clearly, definitely, and explicitly require it, and that is from Midwest Dairy Products. Because the insurers did not clearly, definitely, and explicitly draft their internal statute of limitations to run from the inception of the loss, they do not. Defendant's agent listed the period of exposure on the loss notice form as September 15, 2017 to April 30, 2019. The damage occurred until at least April 30, 2019. Thus, the statute of limitations did not expire until April 30, 2021. Naperville filed this action on April 26, 2021, before the statute of limitations expired. Counsel, does that mean that since there were events that occurred, like rain events, and events that, does that mean that you get to wait until the last rain event that causes an intrusion before you file, do you do anything to assert your claim? According to the policy language, Your Honor, that's the way it's drafted, that you get to wait until after the conclusion of the loss, and as the only cases interpreting this language anywhere that we have found in the country have found that it's at the conclusion of the loss. Counsel, what do you consider the loss, the damage that is done by the water, or each event that it rains over an inch? The cumulative effect of the damage to the hotel is a loss, Your Honor. So you believe that you could turn the clock back to the beginning, when the defect was first noticed by your client, I think back in 15? If that was the conclusion of the loss, that's when it could have started. The policy states that you have until two years after the loss occurred, and the loss is a cumulative effect of the damage, which according to the insurer's own agent was at least through April 30, 2019. So the water damage that happened on October of 2017 that was visible to the insured didn't trigger any kind of an obligation to file a claim or file a lawsuit? Not according to the policy language, Your Honor. Policy language, it does not clearly, definitely, and explicitly state that you have to file a claim within two years after the beginning of the loss. Mr. Conner, what does direct damage mean? Direct damage? It would get, Your Honor, the direct damage would be damage to the property. Well, is there a difference between direct damage and damage that's flowing from something else that occurred before? I'm not sure, Your Honor. I don't believe that the policy, this portion of the policy refers to direct damage. I believe it requires suit within, I actually have the specific language for each insurer. Liberty Mutual says a suit must be brought within two years after the date on which the physical damage occurred, and Fireman's says suit must be brought within two years after the date on which the direct physical loss or damage occurred. So they have different languages, but those policy terms are not defined in the policy. And defendants have not cited any case supporting their interpretation of the internal statute limitation. The cases cited by defendants actually show that insurers know how to draft the internal statute limitations to run from the beginning of the damage, and that these insurers simply chose not to. For example, the internal statute limitations in the foam craft case cited by defendants require that the policyholder file suit within 12 months after discovery of the occurrence. There's no discovery trigger in defendants' policies in their statute limitation. Also, the internal statute limitations in DINO cited by defendants require that the policyholder file suit within one year after the inception of the loss, but defendants here chose not to use comparable inception. And as Panorama and Strauss state, there has to be a difference between the use of inception and after the loss. Defendants' internal statute limitations must be interpreted in favor of Naperville's equally reasonable construction that the statute of limitations runs from the end of the damage, which was less than two years before Naperville filed suit. This court should reverse the dismissal of Naperville's case as Naperville complied with the internal statute of limitations. The trial court also erred in determining that defendants were not stopped from asserting their internal statute of limitations. Illinois courts have held that an insurer is stopped from raising a limitations defense where its actions during negotiations lull the policyholder into a false sense of security, causing it to delay the assertion of its rights. Defendants here understood that this is a complex claim with harm emerging over several years. They understood that their policyholders were not withdrawing their claim. So when defendants communicated with Naperville before and after the alleged statute of limitations expired, without even mentioning the statute of limitations, defendants were acknowledging that the policyholder's claim was still active and was not barred by any statute of limitations. In the months leading up to and when the statute of limitations was allegedly going to expire, Liberty Mutual requested additional documents, corresponded with Naperville, and scheduled an inspection of the loss. It never mentioned that a statute of limitations was about to expire. And then after the statute of limitations allegedly expired, Liberty Mutual conducted the inspection and requested additional documents. Liberty Mutual waited until its denial letter on March 4th, 2020, long after it claims the statute of limitations expired, to inform Naperville of its internal statute of limitations. Liberty Mutual's actions before and after the internal statute of limitations had supposedly expired waived this defense and stopped Liberty Mutual from relying on it. Liberty Mutual also was stopped for another reason for raising its statute of limitations defense because it did not include it in its reservation of rights letter. The Illinois Supreme Court has held that the reservation of rights must specifically refer to the policy defense that may be asserted, and that's the standard Mutual v. Lay case. Illinois courts also hold that bare notice of a reservation of rights is insufficient. Liberty Mutual's reservation of rights letter included a long list of policy provisions that Liberty Mutual claimed procured coverage, but it did not list the internal statute of limitations. Because the reservation of rights letter did not make specific reference to the policy defense, which ultimately may be asserted, Liberty Mutual is precluded from relying on the internal statute of limitations. Fireman's Fund also is a stop from asserting the internal statute of limitations defense. Fireman's communicated with its policyholders long after the alleged statute of limitations expired without mentioning the statute of limitations, implying that Naperville's claim was not barred by any statute of limitations. For example, on December 18th, 2019, nearly two months after it claims the statute of limitations expired, fireman's emailed Naperville regarding the claim. Fireman's did not assert or mention any possible internal statute of limitations in this correspondence. To the contrary, it expressly acknowledged that we did not fully disclaim coverage for this loss and went on to acknowledge that there is coverage for the ensuing water damage. Fireman's actions after the internal statute of limitations allegedly expired waived this defense and stopped fireman's from relying on it. The trial court also erred regarding Naperville's notice of claim. Naperville provided reasonable notice when it provided notice of the loss to both insurers in May 2019 at the latest. In May 2019, Naperville put together the cumulative effect of rain damage over a long period and determined- Mr. Conner. Yes, your honor. Stop for a minute, Mr. Conner. Is everybody distracted by the noise in the background? Yes. A little bit. I did hear it, your honor. Do we know where it's coming from? I think it's the court itself and it's muted now. It wasn't before. So I think it's been fixed. Okay. All right. Thank you. I apologize, Mr. Conner. Go ahead. Thank you, your honor. In May 2019, Naperville put together the cumulative effect of rain damage over a long period of time and determined it should file an insurance claim. Naperville provided notice to defendants agent and thus the defendants even earlier in 2019. When notice is sufficient, it's typically a factual determination that should not be made at the pleading stage. But even if this was the right time, defendant's motion is not well taken. The Illinois Supreme Court has identified five factors courts consider in making a fact-specific reasonable notice determination. The specific language of the policy's notice provision, the insured sophistication in commerce and insurance matters, the insured's awareness of an event which may trigger insurance coverage, the insured's diligence in ascertaining whether policy coverage is available, and the presence or absence of prejudice. These factors support reversing the trial court's dismissal. At a minimum, they raise issues of fact that cannot be adjudicated at this early time in the case. First, defendants notice provisions do not provide a specific period for providing notice. Illinois courts interpret such provisions as requiring notice within a reasonable time. Illinois courts have held that longer notice periods than here were reasonable, even up to five years after the injury. Second, the standard used in determining an insured's sophistication is based on his experience not only in the world of commerce, but also insurance. In the Bergland case cited in Naperville's briefing, the policyholder, like Naperville, purchased insurance for his business and does not have, quote, any other experience or education in insurance to qualify him as a sophisticated insured. Naperville is in the hotel business and does not have education or experience in the insurance industry. Third and fourth, Naperville was not aware that an occurrence had taken place until they reported it to the insurers in 2019. Each time the hotel suffered water penetration, Naperville diligently investigated and addressed what they believed to be discrete events. It was not until 2019 that Naperville became aware of the true extent of the continuous damage and reported it to the insurers. Finally, defendants have suffered no prejudice. They did not describe, much less prove, how they have been prejudiced. Even if defendants had alleged specific prejudice, actual prejudice is a fact question not suitable for determination at the motion to dismiss stage. In sum, each of the notice factors shows Naperville's notice was reasonable. Because Naperville complied with both the internal statute of limitations and the notice provisions, Naperville respectfully requests that this court reverse the trial court's order dismissing Naperville's case and send the case back to the trial court for further proceeding. I'd like to reserve any extra time for rebuttal. Thank you, Your Honor. You have five minutes for rebuttal, Mr. Conner. Thank you very much. Are there any questions for Mr. Conner at this point? I have none. Let's see. I have Mr. Shirl for seven and a half minutes. Mr. Shirley, thank you, Your Honor. Thank you. Thank you. I represent a Liberty Mutual Insurance Company. And if you recall from my briefs, we only insured this hotel from September 15, 2017 to October 31, 2017. It was added as an endorsement later on. So I've got a 47-day period. I think this is pretty simple. Judge Wheaton, below, during our oral argument, said this was not a continuous loss. And this is not a continuous loss. She said there are certainly discrete incidents that happened that create the situation of water intrusion, but it's not continuous. And Justice Albrecht, I think, hit the point correctly. If you look at their second amended complaint, they have very specific dates where there was water damage and water intrusion. So focus on October 13B of their second amended complaint. On October 11, 2017 and October 14, 2017, NOAA recorded precipitation levels of 1.18 and 3.9 inches, respectively, in DuPage County. And then they plead from October 14, 2017 through October 17, 2017, so a period of three days, the hotel experienced water intrusion in the dining area, in the bar, in the hallways, in the stairwells, and in multiple rooms. So if you look at the policy language, that's when the physical damage occurred. And they have two years from that date to bring suit, and they didn't do it. So they specifically allege water damage in the dining area, the bar, hallways, stairwells, multiple rooms. This water damage was not hidden from them. You don't have to be a lawyer. You don't have to be an insurance broker to know if you own a hotel and you got water coming in your dining room, in your hallways, in your rooms, you got to call an insurance person, got to call your insurance broker. You got to put them on notice. Like I said, you don't need to be a lawyer. You don't need to be an insurance person. If I had water coming in in my kitchen, I wouldn't wait 19 months like they did to provide notice. So they filed suit, and it's clearly too late. Council cites two out-of-state cases that don't apply here at all. First, they're obviously not Illinois cases. So let's look at the Panorama case first from Washington. That involved a condominium complex, and the policy covered collapse due to hidden decay. Collapse due to hidden decay. So the question was this. The limitation period said that suit had to be brought within one year after the loss occurred, and the court said, when did the loss occur? It's either the date of the actual collapse, right, is one thing, so you actually see the damage, or the date that the decay that poses a risk of collapse is no longer hidden. So either when you see the actual collapse, physical loss, or you see the hidden decay. That's when the period starts, okay? Our case is completely different. They had heavy rains and water intrusion in October 2017. Those were specific dates. They knew the damage was there. They even admit in their brief that each time that there were these water events, they diligently investigated and addressed what they believed to be discrete events. That's page 26 of their brief. So they knew about the rains. They knew about the water damage that occurred in October 2017. The panoramic case is completely different. The Strauss case from Wisconsin is different. That involved a home. From the time the home was completed, there were latent defects in that home that allowed water infiltration. The insured in that case first learned of water damage.
in October 2010, okay, they submitted a notice of claim in December 2010, two months after they found the, knew about the water damage, and they filed suit October 2011 within the one-year period. So that case is different from our case, and it's not Illinois law in any of it. Let's talk a little bit about the waiver and estoppel arguments. If you look at pages nine and ten of my brief, remember in the summer of 2019, the insured had a lawyer. It was not, it was not Mr. Conner, it was a different lawyer, Mr. Jeffrey Youngerman. They also had a claims advocate in the summer of 2019. So, you know, it's not like the insured was not represented in the summer of 2019. So keep that in mind. The first argument is waiver. I think their argument is we didn't specifically set out the suit limitation provision in our reservation of rights letter or every email or every letter that we saw, that we sent them or every email. The bottom line is this, we are not obligated to, there's no Illinois law or any law that I know of, that says an insurance company is obligated in a reservation of rights letter or every email to keep reminding the insured, oh, by the way, this is a suit limitation provision. The phone craft case from Illinois that I cite, it states the insured cannot blame the insurance company for its failure to read the policy to discover the requirements for bringing suit. It is not the duty of the insurer to inform the insured of the duty. The only regulation I know of in Illinois that talks about suit limitation is section 919.80 of the Illinois Administrative Code. And that requires us to, when we deny coverage, to set out the number of days told. That's the only regulation I know of. And if the Illinois wanted, the state of Illinois wanted insurance companies to set forth the suit limitation provision in a reservation of rights letter, they would have said so. Very quickly on estoppel, I know I have a minute, they claim that we conducted an investigation and asked for additional documents. Here's what happened. In September when the investigation took place, we still didn't know. We still didn't know when the suit limitation had expired and also the additional documents we requested. We were waiting on their attorney to send us an expert report. That's all we were waiting on. Thank you. Thank you, Mr. Shirley. Mr. Fishman? I may please the court. I'm Howard Fishman. I represent the other defendant at the Lee Fireman's Fund. This case was dismissed on the pleadings. The third amended complaint is the operative pleading as it respects my client. So it was the fourth attempt for the hotel to state a claim. And the allegations couldn't be clearer in the complaint and it takes them out of coverage. And it's worth reading verbatim paragraph 10 of the third amended complaint. Since the hotel opened in July 2015, during and after rain events, rain penetrates the exterior walls, windows, and roof of the hotel. The rain penetration and resulting damages have continued through the years up to the present. As the Illinois Supreme Court has held in cases like Zurich versus Rainmark, damages can continue through and trigger several periods. Paragraph 11. These rain events caused damage to the hotel, including the exterior and interior of the hotel. And Mr. Conner concedes in this oral argument, as he must, that the loss here is cumulative. These rain events started in 2015. They continued unabated apparently. There was visible damage. They were placing trash cans underneath these leaks, claiming that that damaged the hotel's reputation. It was visible to hotel guests. That's paragraph 14 of their third amended complaint. And yet, during 2015, 2016, 2017, 2018, until May of 2019, they didn't provide notice to firements. Arguably, they provided notice a few months earlier to Mesero, who they claim is our agent. We dispute that. But as the court well knows on a motion to dismiss, we will deem that true. That's still 41 months that went by. And what's critical in this case that distinguishes it from all of the other cases relied on by the hotel is it's not latent. This was readily apparent water damage that continued. It was direct damage to property. I think Justice McDade asked about that. That's the insuring agreement. Direct loss or damage to property. That's physical. No question. The interpretation of a policy, the rules of construction are well established in Illinois. Insurance policy must be construed as a whole, giving effect to every provision. We cited Lavorsi for that proposition. We cited Central Illinois Light, both Supreme Court cases. The interpretation that's being urged on the court is it's not reasonable. I would say it's absurd, but it is not reasonable. Would lead to absurd results. Absurd results in the sense that they could sit on their rights during all of these ensuing years while water is continuing to infiltrate the hotel and say, well, we didn't have to tell you. We're just waiting for it to stop. That's not good policy. It's not what the insurance policy says, and it's not reasonable notice under the Lavorsi factor. Let me run through those really quick. He did recite the proper test that Lavorsi employs. Prompt notice is deemed reasonable, is construed as reasonable notice. Here we've got 41 months. I don't think that there can be a serious dispute that that is unreasonable as far as time is concerned. The insured sophistication in commerce and insurance, Mr. Shirley correctly brought up that there was a insurance coverage attorney that said C-2900 through 01 in the record and C-5670 V2. There's an affidavit from that lawyer in the record of this case. The third factor is their awareness of the event triggering coverage. We covered that. It was visible remediation to the extent there was any remediation. Trash cans, visible leaks. Their diligence. They say that their diligence is governed by quote actionable occurrence and quote test. There's no support for that in Illinois law. I don't know what it means. It's not in the policy. It's not under Illinois case law. In fact, it was directly, not that explicit language, but that argument was addressed by the Breckenridge case, which is 29, 2019 U.S. District Lexus 233-220. And I'm quoting from or paraphrasing from page 11 of that decision. Breckenridge, who was the policyholder there, argues that the central focus should be on whether a claim existed necessitating a claim for such coverage and did so after producing damage estimates and concluding that the extent of the damage merited submitting a claim. Court goes on to reject that and says that in order to make a subjective determination before deciding whether a claim is quote worth it constitutes a reasonably diligent determination of coverage, the court is not willing to endorse that proposition. So the late notice here is apparent. There's certainly prejudice to Fireman's Fund. It was deprived of 41 months to investigate this loss and if it was covered to address the loss so it wouldn't exacerbate and cause additional property damage and it was deprived of that opportunity and prejudice. Our alternative argument is on the two-year suit limitation. Mr. Connor incorrectly refers to that as an internal statute of limitations. That's not what it is. It is a contractual provision that's a condition proceeding to coverage. It was not met here. We have the same triggering event in July of 2015. The suit was filed five years and nine months later. April 26th of 21 was when this case was initiated. That is not even a close call on the two-year limitations period. It's not a reasonable construction and just to support the condition proceeding argument we cited Hoover and Fomecraft. I'm out of time. No, you have another minute. Oh, okay. I have another minute. Actual knowledge. The Hoover and the Kramer cases stand for the proposition that actual knowledge triggers the suit limitation provision and that only makes common sense. The court shouldn't wear blinders and ignore common sense. The cases that Mr. Connor cites, aside from not being binding and aside from being factually distinguishable, there was no known loss that would have triggered the time limitation for providing reasonable notice or for complying with the suit limitation clause. For that reason, we ask that the trial court be affirmed in both respects. We make that argument any alternative, so either argument or both arguments would carry the day for this court to affirm. Thank you very much. Thank you, Mr. Fishman. Mr. Connor, any rebuttal? Yes, Your Honor. I have a few points to make on rebuttal. The overriding point is that the considerations here are the policy language that the insurers drafted and reasonableness. The only cases cited by any party interpreting this language, and I fully admit that they are out-of-state cases, but they are the only two cases in the whole country interpreting this language, and they both found that the statute of limitations did not begin to run until the conclusion of the loss. These cases were handed down in 2001 for Panorama and 2014 for Strauss, so they were before any of these five insurance policies at issue here were written. The insurers are well aware of this case law holding this. The only cases anywhere in the country interpreting this language. Are they based on the statutes or laws of the states that they cover? They are based on the internal statute of limitations language in each of those policies. Okay. In the Panorama and Strauss cases, they are both internal statute of limitations provisions. And regarding whether this is a continuous loss or not, the specific dates of loss are listed in the complaint as examples only. They are not the only specific dates that there was rain or damage occurring during the policy period. It mentions explicitly in the complaint that they were examples only. Water intrusion damage is not instantaneous or harmful over time in every instance. For example, it could dry out. It may not lead to damage. That is why it wasn't until 2019 that the policyholder was able to assess the total extent of the cumulative damage. Also, water intrusion itself is not damage or loss or direct physical loss. The damage loss or direct physical loss is the damage to the property, damage to the hotel, damage to property inside the hotel. It's not the actual water intrusion itself. So the fact that it might have occurred five different examples over five years, that is not the trigger of the statute of limitations or the notice provision. And also, regarding whether there's any Illinois law regarding including an internal statute of limitations in your Reservation of Rights letter, I've cited specific Illinois Supreme Court precedent holding that you must include any policy defense that you are going to assert to deny a claim in your Reservation of Rights letter, and Liberty Mutual did not do that. They provided a long list, multiple pages of policy defenses that they were going to assert or that they could assert, but they specifically did not mention the statute of limitations. It is also not just, it's not just not telling the policyholder about the statute of limitations, it's also continuing to investigate the claim long after the statute of limitations has allegedly expired that tells the insurer that the statute of limitations does not apply here. If the insurer is continuing to investigate the claim, ask for additional details, come to the site to inspect the hotel to see if there is covered damage, and also sending communications saying that there is coverage for the claim after they have said that the statute of limitations has expired. And I'm also, regarding the 41 months, I'm not sure what that refers to. The defendant's agent listed the period of exposure through at least April 30, 2019, and notice was provided shortly thereafter in May 2019. So there's, it's not a 41-month delay. The cumulative effect of the loss was not determined until then, and that's when notice was provided. If there are no further questions, that is all I have for today. Thank you, Your Honor. Are there any further questions for Mr. Conner? I have none. None for me either. Okay, thank you, Mr. Conner. We thank the three of you for your arguments this afternoon. We'll take the matter under advisement, and we'll issue a written decision as quickly as possible.

In [39]:
system_prompt = "You are a helpful court reporter assistant. Your task is to correct the formatting in an oral argument transcription. Add an introductory header and sub-headers in the style of a legal case brief."

def generate_corrected_transcript(temperature, system_prompt):
    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": transcription,
            }
        ]
    )
    return completion.choices[0].message.content

corrected_text = generate_corrected_transcript(0, system_prompt)

In [40]:
Markdown(corrected_text)

# Case Brief: Naperville Hotel Partners, LLC, and Superhost Hospitality, Inc. v. Liberty Mutual Fire Insurance Company and Fireman's Fund Insurance Company

## Case Number
322-0440

## Parties
- **Appellants**: Naperville Hotel Partners, LLC, and Superhost Hospitality, Inc.
- **Appellees**: Liberty Mutual Fire Insurance Company and Fireman's Fund Insurance Company

## Introduction
This case involves an appeal by Naperville Hotel Partners, LLC, and Superhost Hospitality, Inc. (collectively, "Naperville") against Liberty Mutual Fire Insurance Company and Fireman's Fund Insurance Company (collectively, "Defendants") regarding the dismissal of Naperville's claim for water damage at their hotel property. The appeal addresses the interpretation of the internal statute of limitations in the insurance policy, the estoppel of Defendants from asserting the statute of limitations defense, and the adequacy of Naperville's notice of claim.

## Background
Naperville filed a lawsuit against Defendants seeking coverage for water damage sustained by their hotel property. Defendants moved to dismiss the case on the grounds that Naperville failed to file the lawsuit within the internal statute of limitations period specified in the insurance policies and failed to provide timely notice of the claim. The trial court dismissed Naperville's case, leading to this appeal.

## Arguments

### Appellants' Argument
Colin Connor, representing the Appellants, argued that:
1. The internal statute of limitations should be interpreted to begin running from the end of the injury, not the beginning, based on the policy language and relevant case law.
2. Defendants should be estopped from asserting the statute of limitations defense due to their actions during negotiations, which led Naperville to believe that their claim was still under consideration.
3. Naperville provided adequate notice of its claim within a reasonable time after understanding the cumulative effect of the water damage.

### Appellees' Argument
Representatives for the Appellees argued that:
1. The lawsuit was filed beyond the two-year statute of limitations period as the damage was apparent and not continuous, making the filing untimely.
2. The waiver and estoppel arguments are unfounded as there is no obligation for insurers to remind the insured of the limitations period in every communication.
3. Naperville's notice of the claim was unreasonably delayed, and the policy's requirement for prompt notice was not met.

## Conclusion
The appellate court is tasked with determining whether the trial court erred in dismissing Naperville's case based on the interpretation of the internal statute of limitations, the application of estoppel principles, and the adequacy of notice provided by Naperville. The court will issue a written decision after taking the matter under advisement.

## Decision Pending
The appellate court has not yet issued a decision.